# Descrição experimentos usando NETNS (isolamento de interfaces usando ambientes de rede virtualizados - namespaces)

- Objetivo configurar um script para automatizar tudo.

- Ref ( muito bom inclusive ):  https://www.youtube.com/watch?v=jhk7y7mYI6s&ab_channel=TechUpskill

# COMO RODAR ?

- execute `sh setup_5dominios.sh`

- então, apos inicializar os controladores, sincronize os switches com os comandos:

`sudo ovs-vsctl set-controller switch1 tcp:172.16.1.10:6653` 
`sudo ovs-vsctl set-controller switch2 tcp:172.16.2.10:6653` 
`sudo ovs-vsctl set-controller switch3 tcp:172.16.3.10:6653` 
`sudo ovs-vsctl set-controller switch4 tcp:172.16.4.10:6653` 
`sudo ovs-vsctl set-controller switch5 tcp:172.16.5.10:6653`

# COMO FINALIZAR ?

- execute `sh end_experiments.sh`



# Criar um namespace para cada domínio.

<!-- sudo ip link add <nome-interface> type dummy # isso nao funcionou direito, infelizmente -->

* listar namespaces ativos:
    `sudo ip netns list`

* Criar namespace:
    `sudo ip netns add VRF1`    

* Criando interfaces para utilizar com os namespaces
    `sudo ip link add veth1 type veth peer name veth2`


* Mover interface para namespace:
    `sudo ip link set veth1 netns VRF1`


* Executar comando a partir da namespace (ex: ping, /bin/bash, ifconfig):
    `sudo ip netns exec VRF1 <comando>`

* Subir as interfaces dentro dos namespaces:
    `sudo ip netns exec VRF1 ifconfig veth1 up`

* Criar ovs bridge:
    `sudo ovs-vsctl add-br switch1`

* Adicionar as interfaces criadas:
    `sudo ovs-vsctl add-port switch1 veth2`

* Se não conseguir ver as interfaces, ver com:
    `sudo ip link`

* Associar ips para as interfaces:
    `sudo ifconfig veth1 10.0.0.5/24`

* Rodar terminal a partir do namespace:
    `sudo ip netns exec VRF1 /bin/bash`

* Ou melhor ainda (igual no mininet):
    `sudo ip netns exec VRF1 xterm`

* Ver largura de banda do enlace (interface):
    `tc qdisk show dev`

* Verificar se as filas foram criadas na interface VETH2:
`sudo ovs-appctl qos/show veth2`

O que falta para funcionar !...! conseguir comunicação entre o switch ovs e o controlador
-> Falta um par de interfaces no parent namespace, onde um se conecta com o switch e o outro tem um ip de rede.
-> ai tem que configurar as rotas no virtual namespace e no parent namespace [Feito]

# Realizando experimentos:

O que fazer ... ?

[1] Arrumar as rotas - é o que falta para que os experimentos funcionem [ok-nao testado]

[1.2] Arrumar as regras de fluxo padrão para poder conectar o switch ao controlador [fazendo]

As regras são essas (adaptando para o contexto):
sudo ovs-ofctl add-flow switch1 arp,nw_dst=172.16.1.10,actions:outport=2 # alcancar controlador(flowpri)
sudo ovs-ofctl add-flow switch1 ip,nw_dst=172.16.1.10,actions:outport=2 # alcancar controlador(flowpri)
sudo ovs-ofctl add-flow switch1 icmp,nw_dst=172.16.1.10,actions:outport=2 # alcancar controlador(flowpri)
sudo ovs-ofctl add-flow switch1 arp,nw_dst=172.16.1.50,actions:outport=3
sudo ovs-ofctl add-flow switch1 icmp,nw_dst=172.16.1.50,actions:outport=3
sudo ovs-ofctl add-flow switch1 ip,nw_dst=172.16.1.50,actions:outport=3

No entanto, o controlador trata isso, só precisa adicionar o switch host nas rotas (.50) --> sim mas, ai precisaria modificar o switch features de novo,... vamos colocar no script

[1.3] adicionar a rota default nos hosts

[2] temos que revisar o funcionamento do flowpri, se ele esta rodando corretamente

[2.1] as interfaces tem um nome e as portas associadas ao openflow sao outras, tem que fazer um mapeamento para configurar as filas automáticamente (adicionar interface name nas configuracoes das portas e pegar o nome a partir disso) [OK]

[2.2] Criar pasta para as classificacoes


[3] verificar se todos os domínios estão funcionando e configurados corretamente.

[4]